## Import libraries

In [1]:
import math
import pandas as pd
import matplotlib.pyplot as plt

## Important functions for calculations

### Trigonometry functions in degree

In [2]:
def sind(t):
    return math.sin(math.radians(t))
def cosd(t):
    return math.cos(math.radians(t))
def tand(t):
    return math.tan(math.radians(t))
def asind(t):
    return math.degrees(math.asin(t))
def acosd(t):
    return math.degrees(math.acos(t))
def atand(t):
    return math.degrees(math.atan(t))

In [3]:
# Declitaion angle
def decl(n):
    de = 23.45*sind((360*(284+n))/365)
    return de
# Hour angle
def ha(st):
    return 15*(12-st)

#Theta
def theta(beta , delta , gamma , omega , phi):
    ca = sind(phi)*(sind(delta)*cosd(beta)+cosd(delta)*cosd(gamma)*cosd(omega)*sind(beta)) + cosd(phi)*(cosd(delta)*cosd(omega)*cosd(beta)-sind(delta)*cosd(gamma)*sind(beta)) + cosd(delta)*sind(gamma)*sind(omega)*sind(beta)
    return acosd(ca)

#Theta Z
def theta_z(delta , gamma , omega , phi):
    return theta(0, delta , gamma ,omega ,phi)

#Tilt factor beam
def rb(n, st , beta ,gamma, phi):
    ca = cosd(theta(beta, decl(n), gamma, ha(st),phi))
    caz = cosd(theta_z(decl(n), gamma, ha(st),phi))
    rb = ca/caz
    return rb

#Tilt factor diffuse
def rd(beta):
    return 0.5*(1+cosd(beta))

#Tilt factor reflected
def rr(beta, rohg):
    return (rohg/2)*(1-cosd(beta))

#Theta dash
def thetadash(ng , na , theta):
    return asind(sind(theta)*(na/ng))

#Total solar radiation flux
def I_total(Ibh,Idh , rb , rd , rr):
    return (Ibh*rb + Idh*rd + (Ibh+Idh)*rr)

def roh1(theta , thetad):
    return pow(sind(thetad-theta),2)/pow(sind(thetad+theta),2)
def roh2(theta , thetad):
    return pow(tand(thetad-theta),2)/pow(tand(thetad+theta),2)
def tau1(roh1 , m):
    return (1-roh1)/(1+roh1*(2*m-1))
def tau2(roh2 , m):
    return (1-roh2)/(1+roh2*(2*m-1))
def tau_a(theta_d,k,delC):
    return math.exp(-(k*delC)/cosd(theta_d))
def alpha(theta):
    if theta-85 > 5:
        return 0
    up = 0.0106944*(theta-90)*(1-0.254*pow((cosd(theta)/cosd(85)),4*(theta-85)))
    down = pow((theta/95.406),0.9775)-1
    if type(up/down) == complex:
        return 0
    else:
        return up/down
def tau_alp_b(tau,alpha,roh_d):
    return (tau*alpha)/(1-(1-alpha)*roh_d)
def tau_alp_d(theta ,ng,na,m,k,delC,roh_d):
    td = thetadash(ng,na,theta)
    roh_1 = roh1(theta,td)
    roh_2 = roh2(theta,td)
    tau_1 =  tau1(roh_1,m)
    tau_2 =  tau2(roh_2,m)
    taua = tau_a(td,k,delC)
    alp = alpha(theta)
    return tau_alp_b((taua*0.5*(tau_1+tau_2)),alp,roh_d)
def S(Ibh, Idh, rb, rd, rr, tau_b , tau_d):
    return (Ibh*rb*tau_b + (Idh*rd+(Ibh+Idh)*rr)*tau_d)
def fin_eff(U,K,cpst,L):
    m = pow((U/(K*cpst)),0.5)
    return ((math.exp(m*L)-math.exp(-m*L))/(math.exp(m*L)+math.exp(-m*L)))/(m*L)
def collector_eff(U,W,L,F,Do,cb,Di,hci):
    up = 1/U
    down1 = 1/(U*(2*L*F+Do))
    down2 = 1/cb
    down3 = 1/(hci*math.pi*Di)
    return up/(W*(down1+down2+down3))
def hear_removal_factor(mw,cw,U,fd):
    return (mw*cw/U)*(1-math.exp((-U*fd)/(mw*cw)))

In [4]:
st = [6,7,8,9,10,11,12,13,14,15,16,17,18]
st_t = [6,7,8,9,10,11,12,13,14,15,16,17,18,'Daily']
I = [
    [[0,0,147,300,426,509,537,509,426,300,147,0,0],[0,0,56,77,91,98,98,98,91,77,56,0,0]],
    [[0,42,216,391,502,593,621,593,502,391,216,42,0],[0,25,70,91,98,105,105,105,98,91,70,28,0]],
    [[0,105,293,453,593,649,712,649,593,453,293,105,0],[0,49,84,98,105,105,105,105,105,98,84,49,0]],
    [[0,160,356,523,663,747,767,747,663,523,356,160,0],[0,70,91,98,105,112,112,112,105,98,91,70,0]],
    [[42,209,398,565,705,788,816,788,705,565,398,209,42],[42,77,98,105,105,112,112,112,105,105,98,77,42]],
    [[63,230,412,586,705,788,816,788,705,586,412,230,63],[49,77,98,105,112,112,112,112,112,105,98,77,49]],
    [[56,223,405,565,684,767,795,767,684,565,405,223,56],[42,77,98,105,105,112,112,112,105,105,98,77,42]],
    [[28,188,363,530,670,753,781,753,670,530,363,188,28],[35,70,91,105,105,112,112,112,105,105,91,70,35]],
    [[0,119,307,474,607,698,726,698,607,474,307,119,0],[0,63,84,98,105,105,112,105,105,98,84,63,0]],
    [[0,49,223,384,516,593,628,593,516,384,223,49,0],[0,35,77,91,98,105,105,105,98,91,77,35,0]],
    [[0,0,147,300,419,495,523,495,419,300,147,0,0],[0,0,63,84,91,98,98,98,91,84,63,0,0]],
    [[0,0,112,265,391,460,495,460,391,265,112,0,0],[0,0,49,77,91,98,98,98,91,77,49,0,0]]
]

Twi = [14.8, 16.1, 19.4, 22.7, 24.3, 27.5, 28.5, 29.3, 27.9, 26, 21.4, 17.6]
Ta_r = [
    [8.8, 19.7],
    [9.8, 21.2],
    [11.9, 24.4],
    [14.8, 28.5],
    [17.5, 32.2],
    [20.7, 34.9],
    [22, 34.7],
    [22.3, 34.5],
    [20.6, 33.1],
    [18.1, 30.6],
    [14, 25.5],
    [10.3, 21.1]   
]
DR = [98,93,84,71,56,39,23,11,3,0,3,10,21]
avg_d = [17,47,75,105,135,162,198,228,258,288,318,334]
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [5]:
srs = []
sss = []
for i in range(12):
        ws = -acosd(-tand(30)*tand(decl(30*i+15)))
        sr = round(12+ws/15,2)
        ss = round(12-(ws/15),2)
        srs.append(sr)
        sss.append(ss)        

In [8]:
gamma = 20
beta = 40
phi = 30
n_o_c = 1
k = 0.0068
ng = 1.526
na = 1
delC = 4
roh_g = 0.2
U = 5
K = 385
cpst = 0.5/1000
Do= 11/1000
Di= 10/1000
cb = 35
W = 150/1000
mw = 0.015
cw = 4180
hci = 300
twh=55
vw = 220
Twh=55
thta_diff = 20
if n_o_c == 1 :
    roh_d = 0.15
elif n_o_c == 2:
    roh_d = 0.22
elif n_o_c == 3: 
    roh_d = 0.24

# Working on average days

In [18]:
m = int(input("Enter the month\n"))
gamma = input("Enter value of gamma (Default = 20)\n")
if gamma == "":
    gamma = 20
else: 
    gamma = int(gamma)
n = avg_d[m-1]
print("Day number =" , n)

Enter the month
2
Enter value of gamma (Default = 20)

Day number = 47


In [ ]:

for x in range(12):
    m = x+1
    I_total_m = []
    S_m = []
    Q_m = []
    Ac_m =[]
    NOT_m =[]
    Eff_m =[]
    for i in range(10):
        beta = 10*i
        theta_in = []
        theta_z_in = []
        rb_in = []
        I_total_in = []
        thetadash_in = []
        roh1_in = []
        roh2_in = []
        tau1_in = []
        tau2_in = []
        tau_r = []
        tau_a_in = []
        tau = []
        alpha_in = []
        tau_a_b_in = []
        S_in = []
        qu = []

        # DIFFUSE
        rd_in = rd(beta)
        rr_in = rr(beta, roh_g)
        tau_a_d_in = tau_alp_d(thta_diff, ng, na, n_o_c, k, delC, roh_d)

        # TANK
        L = (W-Do)/2
        F = fin_eff(U,K,cpst,L)
        F_d = collector_eff(U,W,L,F,Do,cb,Di,hci)
        FR = hear_removal_factor(mw,cw,U,F_d)
        for i in range(len(st)):
            theta_in.append(theta(beta,decl(n),gamma,ha(st[i]),phi))
            theta_z_in.append(theta_z(decl(n),gamma,ha(st[i]),phi))
            if rb(n,st[i],beta ,gamma, phi) < 0:
                rb_in.append(0)
            else:
                rb_in.append(rb(n,st[i],beta ,gamma, phi))
            I_total_in.append(I_total(I[m-1][0][i],I[m-1][1][i],rb_in[i],rd(beta),rr(beta,0.2)))
            thetadash_in.append(thetadash(ng,na,theta_in[i]))
            roh1_in.append(roh1(theta_in[i],thetadash_in[i]))
            roh2_in.append(roh2(theta_in[i],thetadash_in[i]))
            tau1_in.append(tau1(roh1_in[i],n_o_c))
            tau2_in.append(tau1(roh2_in[i],n_o_c))
            tau_r.append((tau1_in[i]+tau2_in[i])*0.5)
            tau_a_in.append(tau_a(thetadash_in[i],k,delC))
            tau.append(tau_a_in[i]*tau_r[i])  
            alpha_in.append(alpha(theta_in[i]))
            tau_a_b_in.append(tau_alp_b(tau[i],alpha_in[i],roh_d))
            S_in.append(S(I[m-1][0][i],I[m-1][1][i],rb_in[i], rd_in, rr_in, tau_a_b_in[i], tau_a_d_in))
        ta = Ta_r[m-1][1] - DR[0]*(Ta_r[m-1][1]-Ta_r[m-1][0])/100
        tw_n = [Twi[m-1]]
        if FR*(S_in[0]-U*(Twi[m-1]-ta)) < 0 :
            qu.append(0)
        else:
            qu.append(FR*(S_in[0]-U*(Twi[m-1]-ta)))

        for i in range(1,len(st)): 
            two = tw_n[i-1]+(qu[i-1]/(mw*cw))
            tw_n.append((two+tw_n[i-1])/2)
            ta = Ta_r[m-1][1] - DR[i]*(Ta_r[m-1][1]-Ta_r[m-1][0])/100
            if FR*(S_in[i]-U*(tw_n[i]-ta)) > 0: 
                qu.append(FR*(S_in[i]-U*(tw_n[i]-ta)))
            else:
                qu.append(0)
        I_total_daily = sum(I_total_in[1:len(I_total_in)])*(3600/pow(10,6))
        S_daily = sum(S_in[1:len(S_in)])*(3600/pow(10,6))
        Q_daily = sum(qu[1:len(qu)])*(3600/pow(10,6))
        I_total_in.append(I_total_daily)
        S_in.append(S_daily)
        qu.append(Q_daily)
        Ac = (vw*cw*(Twh-Twi[m-1]))/(Q_daily*pow(10,6))
        Ac_m.append(Ac)
        NOT_m.append(int((Ac/2)/W))
        Eff_m.append(Q_daily/I_total_daily)
        I_total_m.append(I_total_in)
        S_m.append(S_in)
        Q_m.append(qu)    

In [12]:
I_total_m = []
S_m = []
Q_m = []
Ac_m =[]
NOT_m =[]
Eff_m =[]
for i in range(10):
    beta = 10*i
    theta_in = []
    theta_z_in = []
    rb_in = []
    I_total_in = []
    thetadash_in = []
    roh1_in = []
    roh2_in = []
    tau1_in = []
    tau2_in = []
    tau_r = []
    tau_a_in = []
    tau = []
    alpha_in = []
    tau_a_b_in = []
    S_in = []
    qu = []

    # DIFFUSE
    rd_in = rd(beta)
    rr_in = rr(beta, roh_g)
    tau_a_d_in = tau_alp_d(thta_diff, ng, na, n_o_c, k, delC, roh_d)

    # TANK
    L = (W-Do)/2
    F = fin_eff(U,K,cpst,L)
    F_d = collector_eff(U,W,L,F,Do,cb,Di,hci)
    FR = hear_removal_factor(mw,cw,U,F_d)
    for i in range(len(st)):
        theta_in.append(theta(beta,decl(n),gamma,ha(st[i]),phi))
        theta_z_in.append(theta_z(decl(n),gamma,ha(st[i]),phi))
        if rb(n,st[i],beta ,gamma, phi) < 0:
            rb_in.append(0)
        else:
            rb_in.append(rb(n,st[i],beta ,gamma, phi))
        I_total_in.append(I_total(I[m-1][0][i],I[m-1][1][i],rb_in[i],rd(beta),rr(beta,0.2)))
        thetadash_in.append(thetadash(ng,na,theta_in[i]))
        roh1_in.append(roh1(theta_in[i],thetadash_in[i]))
        roh2_in.append(roh2(theta_in[i],thetadash_in[i]))
        tau1_in.append(tau1(roh1_in[i],n_o_c))
        tau2_in.append(tau1(roh2_in[i],n_o_c))
        tau_r.append((tau1_in[i]+tau2_in[i])*0.5)
        tau_a_in.append(tau_a(thetadash_in[i],k,delC))
        tau.append(tau_a_in[i]*tau_r[i])  
        alpha_in.append(alpha(theta_in[i]))
        tau_a_b_in.append(tau_alp_b(tau[i],alpha_in[i],roh_d))
        S_in.append(S(I[m-1][0][i],I[m-1][1][i],rb_in[i], rd_in, rr_in, tau_a_b_in[i], tau_a_d_in))
    ta = Ta_r[m-1][1] - DR[0]*(Ta_r[m-1][1]-Ta_r[m-1][0])/100
    tw_n = [Twi[m-1]]
    if FR*(S_in[0]-U*(Twi[m-1]-ta)) < 0 :
        qu.append(0)
    else:
        qu.append(FR*(S_in[0]-U*(Twi[m-1]-ta)))

    for i in range(1,len(st)): 
        two = tw_n[i-1]+(qu[i-1]/(mw*cw))
        tw_n.append((two+tw_n[i-1])/2)
        ta = Ta_r[m-1][1] - DR[i]*(Ta_r[m-1][1]-Ta_r[m-1][0])/100
        if FR*(S_in[i]-U*(tw_n[i]-ta)) > 0: 
            qu.append(FR*(S_in[i]-U*(tw_n[i]-ta)))
        else:
            qu.append(0)
    I_total_daily = sum(I_total_in[1:len(I_total_in)])*(3600/pow(10,6))
    S_daily = sum(S_in[1:len(S_in)])*(3600/pow(10,6))
    Q_daily = sum(qu[1:len(qu)])*(3600/pow(10,6))
    I_total_in.append(I_total_daily)
    S_in.append(S_daily)
    qu.append(Q_daily)
    Ac = (vw*cw*(Twh-Twi[m-1]))/(Q_daily*pow(10,6))
    Ac_m.append(Ac)
    NOT_m.append(int((Ac/2)/W))
    Eff_m.append(Q_daily/I_total_daily)
    I_total_m.append(I_total_in)
    S_m.append(S_in)
    Q_m.append(qu)    

## Putting data in data frames

In [13]:
I_avg = pd.DataFrame(data= {
    "ST":st_t,
    "Beta = 0" : I_total_m[0],
    "Beta = 10" : I_total_m[1],
    "Beta = 20" : I_total_m[2],
    "Beta = 30" : I_total_m[3],
    "Beta = 40" : I_total_m[4],
    "Beta = 50" : I_total_m[5],
    "Beta = 60" : I_total_m[6],
    "Beta = 70" : I_total_m[7],
    "Beta = 80" : I_total_m[8],
    "Beta = 90" : I_total_m[9]
})
S_avg = pd.DataFrame(data= {
    "ST":st_t,
    "Beta = 0" : S_m[0],
    "Beta = 10" : S_m[1],
    "Beta = 20" : S_m[2],
    "Beta = 30" : S_m[3],
    "Beta = 40" : S_m[4],
    "Beta = 50" : S_m[5],
    "Beta = 60" : S_m[6],
    "Beta = 70" : S_m[7],
    "Beta = 80" : S_m[8],
    "Beta = 90" : S_m[9],
})
Q_avg = pd.DataFrame(data= {
    "ST":st_t,
    "Beta = 0" : Q_m[0],
    "Beta = 10" : Q_m[1],
    "Beta = 20" : Q_m[2],
    "Beta = 30" : Q_m[3],
    "Beta = 40" : Q_m[4],
    "Beta = 50" : Q_m[5],
    "Beta = 60" : Q_m[6],
    "Beta = 70" : Q_m[7],
    "Beta = 80" : Q_m[8],
    "Beta = 90" : Q_m[9],
})
Dim = pd.DataFrame(data={
    "Tilt Angle" : list(range(0,100,10)),
    "Area of collector" : Ac_m,
    "Number of tubes" : NOT_m,
    "Collector efficiency" : Eff_m
})

In [21]:
print(f"Total solar radiation at {months[m-1]} at different tilt angles")
I_avg

Total solar radiation at February at different tilt angles


,ST,Beta = 0,Beta = 10,Beta = 20,Beta = 30,Beta = 40,Beta = 50,Beta = 60,Beta = 70,Beta = 80,Beta = 90
0,6,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,7,154.000,195.940910,233.140597,264.468765,288.973525,305.910313,314.764511,315.267090,307.402780,291.410533
2,8,377.000,442.332829,496.647243,538.292926,566.004495,578.939948,576.706247,559.371264,527.461712,481.947146
3,9,551.000,625.158295,683.484497,724.206394,746.086672,748.460510,731.255780,694.995239,640.780646,570.259282
4,10,698.000,775.751868,833.648931,869.932017,883.498683,873.936711,841.536639,787.282925,712.824042,620.422384
5,11,754.000,824.196788,873.236950,899.630425,902.575263,881.981984,838.476307,773.380127,688.671361,586.923840
6,12,817.000,880.408051,921.142948,937.966982,930.368963,898.579752,843.565250,766.997044,671.201619,559.089672
7,13,754.000,799.727606,825.042071,829.174226,811.998519,774.036824,716.442590,640.965786,549.899738,446.011441
8,14,698.000,727.433155,738.479644,730.803827,704.638930,660.779958,600.559546,525.807459,438.795002,342.166005
9,15,551.000,560.807229,556.737640,538.914885,507.880500,464.577449,410.321473,346.761113,275.827617,199.676265


In [15]:
print(f"Incident solar flux at {months[m-1]} at different tilt angles")
S_avg

Incident solar flux at March at different tilt angles


,ST,Beta = 0,Beta = 10,Beta = 20,Beta = 30,Beta = 40,Beta = 50,Beta = 60,Beta = 70,Beta = 80,Beta = 90
0,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,7,80.920856,116.911740,153.470391,185.887529,211.693635,229.651550,239.154801,239.947737,232.017015,215.573541
2,8,266.106638,334.481316,391.076100,433.935090,462.160664,475.291765,473.134146,455.728490,423.352275,376.566622
3,9,436.458373,512.143577,570.432263,610.515107,631.837802,634.064591,617.149958,581.347416,527.141344,455.248304
4,10,578.010307,654.848419,711.099091,746.013519,758.914719,749.464261,717.896681,664.790278,590.802017,496.710281
5,11,635.572142,703.849059,751.073492,776.405791,779.001757,758.784402,716.445495,652.724715,568.247228,463.833707
6,12,691.940093,753.388061,792.550089,808.575790,800.912383,769.856343,716.120115,640.382085,543.332211,426.395625
7,13,635.572142,680.140123,704.553677,708.365326,691.483412,654.199478,597.031358,520.628022,426.078970,316.734759
8,14,578.010307,607.263127,618.092693,610.329724,584.071010,539.668069,477.752076,399.535775,308.077662,212.234146
9,15,436.458373,446.550919,442.312126,423.819164,391.444725,345.995455,289.129975,224.431375,159.636567,109.028888


In [16]:
print(f"Rate of useful energy gain at {months[m-1]} at different tilt angles")
Q_avg

Rate of useful energy gain at March at different tilt angles


,ST,Beta = 0,Beta = 10,Beta = 20,Beta = 30,Beta = 40,Beta = 50,Beta = 60,Beta = 70,Beta = 80,Beta = 90
0,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,7,40.516629,71.026164,102.016995,129.497060,151.372940,166.595895,174.651817,175.323991,168.601104,154.661943
2,8,200.897800,257.827913,304.755922,340.158738,363.346203,373.962926,371.861619,357.084095,329.865955,290.676818
3,9,345.402509,406.605535,453.383305,485.236165,501.788314,502.802588,488.262759,458.389642,413.586264,354.438142
4,10,461.668958,521.780649,565.249706,591.644668,600.498439,591.579609,565.109896,521.577975,461.519552,385.552758
5,11,503.866686,554.689269,589.037518,606.417544,606.236295,588.491802,553.785891,502.727519,435.807331,353.658412
6,12,543.096265,586.412136,612.764903,621.668267,612.796255,586.463554,543.269171,483.749523,408.433597,318.445480
7,13,483.314214,510.856819,523.816557,522.065116,505.678433,474.956377,430.312740,372.240870,301.590566,221.080110
8,14,422.421480,436.050458,437.056989,425.552922,401.771378,366.052931,318.893463,261.247564,195.605429,129.260781
9,15,289.739294,286.665417,274.864453,254.653389,226.491349,191.093207,149.809054,105.570559,64.749651,38.993943


In [17]:
print(f"Dimensions and charcterstics at {months[m-1]} at different tilt angles")
Dim

Dimensions and charcterstics at March at different tilt angles


,Tilt Angle,Area of collector,Number of tubes,Collector efficiency
0,0,2.655229,8,0.581967
1,10,2.426674,8,0.591311
2,20,2.298665,7,0.596670
3,30,2.248285,7,0.598818
4,40,2.267164,7,0.598092
5,50,2.357595,7,0.594420
6,60,2.528902,8,0.586631
7,70,2.808545,9,0.576425
8,80,3.271442,10,0.559666
9,90,4.044593,13,0.533109


# Instantaneous analysis

In [ ]:
Q_y = []
for i in range(12):
    m = i+1    
    n = avg_d[m-1]
    Q_m = []
    for i in range(10):
        beta = 10*i
        theta_in = []
        theta_z_in = []
        rb_in = []
        I_total_in = []
        thetadash_in = []
        roh1_in = []
        roh2_in = []
        tau1_in = []
        tau2_in = []
        tau_r = []
        tau_a_in = []
        tau = []
        alpha_in = []
        tau_a_b_in = []
        S_in = []
        qu = []

        # DIFFUSE
        rd_in = rd(beta)
        rr_in = rr(beta, roh_g)
        tau_a_d_in = tau_alp_d(thta_diff, ng, na, n_o_c, k, delC, roh_d)

        # TANK
        L = (W-Do)/2
        F = fin_eff(U,K,cpst,L)
        F_d = collector_eff(U,W,L,F,Do,cb,Di,hci)
        FR = hear_removal_factor(mw,cw,U,F_d)
        for i in range(len(st)):
            theta_in.append(theta(beta,decl(n),gamma,ha(st[i]),phi))
            theta_z_in.append(theta_z(decl(n),gamma,ha(st[i]),phi))
            if rb(n,st[i],beta ,gamma, phi) < 0:
                rb_in.append(0)
            else:
                rb_in.append(rb(n,st[i],beta ,gamma, phi))
            I_total_in.append(I_total(I[m-1][0][i],I[m-1][1][i],rb_in[i],rd(beta),rr(beta,0.2)))
            thetadash_in.append(thetadash(ng,na,theta_in[i]))
            roh1_in.append(roh1(theta_in[i],thetadash_in[i]))
            roh2_in.append(roh2(theta_in[i],thetadash_in[i]))
            tau1_in.append(tau1(roh1_in[i],n_o_c))
            tau2_in.append(tau1(roh2_in[i],n_o_c))
            tau_r.append((tau1_in[i]+tau2_in[i])*0.5)
            tau_a_in.append(tau_a(thetadash_in[i],k,delC))
            tau.append(tau_a_in[i]*tau_r[i])  
            alpha_in.append(alpha(theta_in[i]))
            tau_a_b_in.append(tau_alp_b(tau[i],alpha_in[i],roh_d))
            S_in.append(S(I[m-1][0][i],I[m-1][1][i],rb_in[i], rd_in, rr_in, tau_a_b_in[i], tau_a_d_in))
        ta = Ta_r[m-1][1] - DR[0]*(Ta_r[m-1][1]-Ta_r[m-1][0])/100
        tw_n = [Twi[m-1]]
        if FR*(S_in[0]-U*(Twi[m-1]-ta)) < 0 :
            qu.append(0)
        else:
            qu.append(FR*(S_in[0]-U*(Twi[m-1]-ta)))

        for i in range(1,len(st)): 
            two = tw_n[i-1]+(qu[i-1]/(mw*cw))
            tw_n.append((two+tw_n[i-1])/2)
            ta = Ta_r[m-1][1] - DR[i]*(Ta_r[m-1][1]-Ta_r[m-1][0])/100
            if FR*(S_in[i]-U*(tw_n[i]-ta)) > 0: 
                qu.append(FR*(S_in[i]-U*(tw_n[i]-ta)))
            else:
                qu.append(0)
        Q_m.append(qu)
    Q_y.append(Q_m)

### Plots

In [ ]:
mon = int(input("What month you want to plot?\n"))
plt.plot(st,Q_y[mon-1][0],
         st,Q_y[mon-1][1],
         st,Q_y[mon-1][2],
         st,Q_y[mon-1][3],
         st,Q_y[mon-1][4],
         st,Q_y[mon-1][5],
         st,Q_y[mon-1][6],
         st,Q_y[mon-1][7],
         st,Q_y[mon-1][8],
         st,Q_y[mon-1][9])
plt.legend(["beta=0",
            "beta=10",
            "beta=20",
            "beta=30",
            "beta=40",
            "beta=50",
            "beta=60",
            "beta=70",
            "beta=80",
            "beta=90"])
plt.title(f"Instantaneous analysis for {months[mon-1]} at gamma = {gamma}")
plt.xlabel("Solar time")
print(srs[mon-1])
plt.xlim([srs[mon-1],sss[mon-1]])
plt.ylabel("qu (w/m^2)")
plt.show()

# Daily analysis

In [ ]:
Q_yd = []
for mo in range(12):
    m = mo+1
    n = avg_d[m-1]
    Q_m = []
    for ga in range(10):
        gamma = ga*10
        I_total_g = []
        S_g = []
        Q_g = []
        for be in range(10):
            beta = 10*be
            theta_in = []
            theta_z_in = []
            rb_in = []
            I_total_in = []
            thetadash_in = []
            roh1_in = []
            roh2_in = []
            tau1_in = []
            tau2_in = []
            tau_r = []
            tau_a_in = []
            tau = []
            alpha_in = []
            tau_a_b_in = []
            S_in = []
            qu = []

            # DIFFUSE
            rd_in = rd(beta)
            rr_in = rr(beta, roh_g)
            tau_a_d_in = tau_alp_d(thta_diff, ng, na, n_o_c, k, delC, roh_d)

            # TANK
            L = (W-Do)/2
            F = fin_eff(U,K,cpst,L)
            F_d = collector_eff(U,W,L,F,Do,cb,Di,hci)
            FR = hear_removal_factor(mw,cw,U,F_d)
            for i in range(len(st)):
                theta_in.append(theta(beta,decl(n),gamma,ha(st[i]),phi))
                theta_z_in.append(theta_z(decl(n),gamma,ha(st[i]),phi))
                if rb(n,st[i],beta ,gamma, phi) < 0:
                    rb_in.append(0)
                else:
                    rb_in.append(rb(n,st[i],beta ,gamma, phi))
                I_total_in.append(I_total(I[m-1][0][i],I[m-1][1][i],rb_in[i],rd(beta),rr(beta,0.2)))
                thetadash_in.append(thetadash(ng,na,theta_in[i]))
                roh1_in.append(roh1(theta_in[i],thetadash_in[i]))
                roh2_in.append(roh2(theta_in[i],thetadash_in[i]))
                tau1_in.append(tau1(roh1_in[i],n_o_c))
                tau2_in.append(tau1(roh2_in[i],n_o_c))
                tau_r.append((tau1_in[i]+tau2_in[i])*0.5)
                tau_a_in.append(tau_a(thetadash_in[i],k,delC))
                tau.append(tau_a_in[i]*tau_r[i])  
                alpha_in.append(alpha(theta_in[i]))
                tau_a_b_in.append(tau_alp_b(tau[i],alpha_in[i],roh_d))
                S_in.append(S(I[m-1][0][i],I[m-1][1][i],rb_in[i], rd_in, rr_in, tau_a_b_in[i], tau_a_d_in))
            ta = Ta_r[m-1][1] - DR[0]*(Ta_r[m-1][1]-Ta_r[m-1][0])/100
            tw_n = [Twi[m-1]]
            if FR*(S_in[0]-U*(Twi[m-1]-ta)) < 0 :
                qu.append(0)
            else:
                qu.append(FR*(S_in[0]-U*(Twi[m-1]-ta)))

            for i in range(1,len(st)): 
                two = tw_n[i-1]+(qu[i-1]/(mw*cw))
                tw_n.append((two+tw_n[i-1])/2)
                ta = Ta_r[m-1][1] - DR[i]*(Ta_r[m-1][1]-Ta_r[m-1][0])/100
                if FR*(S_in[i]-U*(tw_n[i]-ta)) > 0: 
                    qu.append(FR*(S_in[i]-U*(tw_n[i]-ta)))
                else:
                    qu.append(0)
            Q_daily = sum(qu[1:len(qu)])*(3600/pow(10,6))
            Q_g.append(Q_daily)   
        Q_m.append(Q_g)      
    Q_yd.append(Q_m)    

### Plots

In [ ]:
mon = int(input("What month you want to plot?\n"))
for i in range(10):  
    plt.plot(
        list(range(0,100,10)),Q_yd[mon-1][i]
    )
plt.xticks(range(0,100,10))
plt.legend(["gamma=0",
            "gamma=10",
            "gamma=20",
            "gamma=30",
            "gamma=40",
            "gamma=50",
            "gamma=60",
            "gamma=70",
            "gamma=80",
            "gamma=90"] , ncol=2)
plt.title(f"Daily analysis for {months[mon-1]}")
plt.xlabel("Tilt angle")
plt.ylabel("qu (MJ/Day.m^2)")
plt.show()

### Find the optimum Beta and Gamma for maximum Q daily

In [ ]:
opt_betas = []
opt_gammas = []
max_q = []
for i in range(12):
    mx = list(map(max,Q_yd[i]))
    gmx = max(mx)
    gam_ind = mx.index(gmx)
    gam_opt = gam_ind*10
    be_opt = Q_yd[i][gam_ind].index(gmx)*10
    opt_betas.append(be_opt)
    opt_gammas.append(gam_opt)    
    max_q.append(gmx)

In [ ]:
Optimum_data = pd.DataFrame(data= {
    "Month":months,
    "Optimum Tilt angle": opt_betas,
    "Optimum Gamma": opt_gammas,
    "Maximum rate of useful energy gain per day" : max_q 
})
Optimum_data.style.set_table_attributes("style = 'font-size:11px; font-weight:700; text-align:center'").set_caption("Optimum Beta and Gamma for maximum Q daily")

# Yearly analysis

In [ ]:
Q_y = []
d_i_m = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31] 
for i in range(10):
    Q_yb = []
    for b in range(10):
        sum = 0
        for m in range(12):
            sum += Q_yd[m][i][b]*d_i_m[m]
        Q_yb.append(sum)
    Q_y.append(Q_yb)

## Plot

In [ ]:
for i in range(10):  
    plt.plot(
        list(range(0,100,10)),Q_y[i]
    )
plt.xticks(range(0,100,10))
plt.legend(["gamma=0",
            "gamma=10",
            "gamma=20",
            "gamma=30",
            "gamma=40",
            "gamma=50",
            "gamma=60",
            "gamma=70",
            "gamma=80",
            "gamma=90"])
plt.title(f"Yearly analysis")
plt.xlabel("Tilt angle")
plt.ylabel("qu (MJ/Day.m^2)")
plt.show()

### Find the optimum Beta and Gamma for maximum Q yearly

In [ ]:
mx = list(map(max,Q_y))
gmx = max(mx)
gam_ind = mx.index(gmx)
gam_opt_y = gam_ind*10
be_opt_y = Q_y[gam_ind].index(gmx)*10   

In [ ]:
gmx

In [ ]:
Optimum_data_y = pd.DataFrame(data= {
    "Optimum Tilt angle": [be_opt_y],
    "Optimum Gamma":[gam_opt_y],
    "Maximum rate of useful energy gain per year" : [gmx]
})
Optimum_data_y.style.set_table_attributes("style = 'font-size:14px; font-weight:700; text-align:center'").set_caption("Optimum Beta and Gamma for maximum Q yearly")


In [ ]:
gamma = 0
beta = 30
phi = 30
n_o_c = 1
k = 0.0068
ng = 1.526
na = 1
delC = 4
roh_g = 0.2
U = 5
K = 385
cpst = 0.5/1000
Do= 11/1000
Di= 10/1000
cb = 35
W = 150/1000
mw = 0.015
cw = 4180
hci = 300
twh=55
vw = 250
Twh=55
thta_diff = 20
if n_o_c == 1 :
    roh_d = 0.15
elif n_o_c == 2:
    roh_d = 0.22
elif n_o_c == 3: 
    roh_d = 0.24

In [ ]:
Q_Months = []
for i in range(12):
    m = i+1
    n = avg_d[m-1]
    theta_in = []
    theta_z_in = []
    rb_in = []
    I_total_in = []
    thetadash_in = []
    roh1_in = []
    roh2_in = []
    tau1_in = []
    tau2_in = []
    tau_r = []
    tau_a_in = []
    tau = []
    alpha_in = []
    tau_a_b_in = []
    S_in = []
    qu = []

    # DIFFUSE
    rd_in = rd(beta)
    rr_in = rr(beta, roh_g)
    tau_a_d_in = tau_alp_d(thta_diff, ng, na, n_o_c, k, delC, roh_d)

    # TANK
    L = (W-Do)/2
    F = fin_eff(U,K,cpst,L)
    F_d = collector_eff(U,W,L,F,Do,cb,Di,hci)
    FR = hear_removal_factor(mw,cw,U,F_d)
    for i in range(len(st)):
        theta_in.append(theta(beta,decl(n),gamma,ha(st[i]),phi))
        theta_z_in.append(theta_z(decl(n),gamma,ha(st[i]),phi))
        if rb(n,st[i],beta ,gamma, phi) < 0:
            rb_in.append(0)
        else:
            rb_in.append(rb(n,st[i],beta ,gamma, phi))
        I_total_in.append(I_total(I[m-1][0][i],I[m-1][1][i],rb_in[i],rd(beta),rr(beta,0.2)))
        thetadash_in.append(thetadash(ng,na,theta_in[i]))
        roh1_in.append(roh1(theta_in[i],thetadash_in[i]))
        roh2_in.append(roh2(theta_in[i],thetadash_in[i]))
        tau1_in.append(tau1(roh1_in[i],n_o_c))
        tau2_in.append(tau1(roh2_in[i],n_o_c))
        tau_r.append((tau1_in[i]+tau2_in[i])*0.5)
        tau_a_in.append(tau_a(thetadash_in[i],k,delC))
        tau.append(tau_a_in[i]*tau_r[i])  
        alpha_in.append(alpha(theta_in[i]))
        tau_a_b_in.append(tau_alp_b(tau[i],alpha_in[i],roh_d))
        S_in.append(S(I[m-1][0][i],I[m-1][1][i],rb_in[i], rd_in, rr_in, tau_a_b_in[i], tau_a_d_in))
    ta = Ta_r[m-1][1] - DR[0]*(Ta_r[m-1][1]-Ta_r[m-1][0])/100
    tw_n = [Twi[m-1]]
    if FR*(S_in[0]-U*(Twi[m-1]-ta)) < 0 :
        qu.append(0)
    else:
        qu.append(FR*(S_in[0]-U*(Twi[m-1]-ta)))

    for i in range(1,len(st)): 
        two = tw_n[i-1]+(qu[i-1]/(mw*cw))
        tw_n.append((two+tw_n[i-1])/2)
        ta = Ta_r[m-1][1] - DR[i]*(Ta_r[m-1][1]-Ta_r[m-1][0])/100
        if FR*(S_in[i]-U*(tw_n[i]-ta)) > 0: 
            qu.append(FR*(S_in[i]-U*(tw_n[i]-ta)))
        else:
            qu.append(0)
    I_total_daily = sum(I_total_in[1:len(I_total_in)])*(3600/pow(10,6))
    S_daily = sum(S_in[1:len(S_in)])*(3600/pow(10,6))
    Q_daily = sum(qu[1:len(qu)])*(3600/pow(10,6))
    Q_Months.append(Q_daily)

In [ ]:
Q_Months

In [ ]:
days_in = [31, 28,31,30,31,30,31,31,30,31,30,31]

In [ ]:
Twh

In [ ]:
monthly_de = []
monthly_s = []
for i in range(12):
    m = i + 1
    demand = vw*cw*(Twh-Twi[m-1])*days_in[i]/1000000
    monthly_s.append(Q_Months[i]*days_in[i]*2.5)
    monthly_de.append(demand)

In [ ]:
monthly_de

In [ ]:
monthly_s

In [ ]:
for i in range(12):
    print(monthly_s[i]-monthly_de[i])